In [ ]:
# Load libraries
import numpy as np
import math as mt
import scipy as sp

In [ ]:
# Define constants
R = sp.constants.R           # Ideal gas constant (J.mol/K)
T = 25 + 273.15              # Temperature (K)
conc = 0.997*1000/18.01528   # Concentration of pure water at 25°C (mol/L)
conv = 2625500.2             # Conversion from 1 Eh to J/mol

# Calculation of the stability constant
Function to compute the solvation free energy $(\Delta_r G)$ and the logarithm of the stability constant $(\log \beta)$ at a given temperature.

In this notebook, 3 calculation methods are used:
- Monomer [1]
- Cluster [2]
- Hess [3]

In [ ]:
def ConstanteK(G_L, G_M, G_ML, G_H2O, G_H2O_n, n, T, method, G_R = 0, G_MR = 0, logK_R = 0):
 #     G_L: Ligand free energy (Eh)
 #     G_M: Cu(H2O)6 complex free energy (Eh)
 #    G_ML: Cu-Ligand complex free energy (Eh)
 #   G_H2O: Water free energy (Eh)
 # G_H2O_n: Water cluster free energy (Eh)
 #       n: Stoichiometric coefficient of water in the products
 #  logK_R: Logarithm of the stability constant of the reference complex (experimentally determined)
 #       T: Reaction temperature
 #  method: 'monomer', 'cluster' or 'Hess'

 # Optional parameters: If an experimental reference complex is available, the 'Hess' method can be used and the G values calculated by DFT can be input.
 #  G_R: Reference ligand free energy (Eh)
 # G_MR: Reference Cu-complex free energy (Eh)

  if(method == 'monomer'):
    # ΔG_aq without correction in J/mol
    Del_G_aq = ((G_ML + n*G_H2O) - (G_M + G_L))*conv
    logK = - Del_G_aq/(mt.log(10)*R*T)

    # Correction factor in logK:
    corr = -((n-1)*np.log(24.46) + n*np.log(conc))/np.log(10)

  if(method == 'cluster'):
    # ΔG_aq without correction in J/mol
    Del_G_aq = ((G_ML + G_H2O_n) - (G_M + G_L))*conv
    logK = - Del_G_aq/(mt.log(10)*R*T)

    # Correction factor in logK:
    corr = -(np.log(conc/n))/np.log(10)

  if(method == 'Hess'):
    # ΔG_aq in J/mol for the auxiliary reaction CuR + L^{2-} -> CuL + R^{2-}
    Del_G_aux = ((G_ML + G_R) - (G_MR + G_L))*conv
    logK = - Del_G_aux/(mt.log(10)*R*T) + logK_R

    # Correction factor (No correction term is applied for this method)
    corr = 0

  # Corrected logK
  logK = logK + corr
  return logK, corr

# Computed free energy database
- Water molecule (xTB, revTPSS, B3LYP, M06)
- Copper hexaaquo complex $[\text{Cu(H2O)}_6]$ (xTB, revTPSS, B3LYP, M06)
- Water cluster $(\text{H}_2\text{O})_6$ (xTB, revTPSS, B3LYP, M06)
- DU complex and ligand (1, 2, 3) (xTB, revTPSS, B3LYP, M06)
- DG complex and ligand (1, 2) (xTB, B3LYP)
- DGB complex and ligand (xTB, B3LYP)
- Reference complex (rtsc, gtsc, ptsm) (xTB, revTPSS, B3LYP, M06)
- Proposed complexes and ligands with conformations (C1 - C8 & C1-OH, C2-OH, C3-OH, C7-OH) (B3LYP)

In [ ]:
# Data using def2-SVP basis and varying the functional:
H2O_B3LYP   = -76.32999861
H2O_M06     = -76.33311223
H2O_revTPSS = -76.33132452
H2O_xTB     = -5.077858752833

Cu_H2O_B3LYP   = -2097.94894161
Cu_H2O_M06     = -2097.96061084
Cu_H2O_revTPSS = -2097.59428975
Cu_H2O_xTB     =  -33.707556648529

H2O_6_B3LYP   = -457.98455078
H2O_6_M06     = -457.98818222
H2O_6_revTPSS = -457.99217898
H2O_6_xTB     = -30.423820068483


# Diego Uehara (DU) Data
DU1_L_revTPSS = -1722.36523467
DU1_L_B3LYP   = -1721.66358747
DU1_L_M06     = -1721.32821243
DU1_L_xTB     = -110.272510878906
DU2_L_revTPSS = -1761.62534652
DU2_L_B3LYP   = -1760.89892147
DU2_L_M06     = -1760.55506237
DU2_L_xTB     = -113.407957326149
DU3_L_revTPSS = -1990.39495807
DU3_L_B3LYP   = -1989.60747966
DU3_L_M06     = -1989.24732295
DU3_L_xTB     = -127.812363078405

DU1_CuL_revTPSS = -3362.09448999
DU1_CuL_B3LYP   = -3361.74976362
DU1_CuL_M06     = -3361.41005220
DU1_CuL_xTB     = -113.663009357611
DU2_CuL_revTPSS = -3401.34780383
DU2_CuL_B3LYP   = -3400.98138129
DU2_CuL_M06     = -3400.63150865
DU2_CuL_xTB     = -116.800171845567
DU3_CuL_revTPSS = -3630.11425904
DU3_CuL_M06     = -3629.32168800
DU3_CuL_xTB     =  -131.203121154232
DU3_CuL_B3LYP   = -3629.68857429

# Deborah Gonzales (DG) complex data
DG1_L_B3LYP = -1832.73227146
DG1_L_xTB   = -115.314835367684
DG2_L_B3LYP = -1793.49645752
DG2_L_xTB   = -112.164549306407

DG1_CuL_B3LYP = -3472.82510699
DG1_CuL_xTB   = -118.684934868650
DG2_CuL_B3LYP = -3433.59402238
DG2_CuL_xTB   = -115.548121549562

# David Grados (DGB) complex data
DGB_L_B3LYP   = -1375.66499902
DGB_L_xTB     = -88.069160470315
DGB_CuL_B3LYP = -3015.74766710
DGB_CuL_xTB   = -91.458165860414


# Experimental data values
 # Exp1: rtsc
 # Exp2: gtsc
 # Exp3: ptsm

# Reference data
Exp1_L_revTPSS = -1548.15853582
Exp1_L_B3LYP   = -1548.08601279
Exp1_L_M06     = -1548.06014516
Exp1_L_xTB     = -53.691492797342
Exp2_L_revTPSS = -1662.54352839
Exp2_L_B3LYP   = -1662.44557435
Exp2_L_M06     = -1662.415279666
Exp2_L_xTB     = -60.893976347704
Exp3_L_revTPSS = -1397.87405102
Exp3_L_B3LYP   = -1397.80038975
Exp3_L_M06     = -1397.76825138
Exp3_L_xTB     = -45.571780476625

Exp1_CuL_revTPSS = -3187.89632475
Exp1_CuL_B3LYP   = -3188.18043689
Exp1_CuL_M06     = -3188.15566046
Exp1_CuL_xTB     = -57.047499040022
Exp2_CuL_revTPSS = -3302.28854559
Exp2_CuL_B3LYP   = -3302.54679773
Exp2_CuL_M06     = -3302.51834717
Exp2_CuL_xTB     = -64.252006023925
Exp3_CuL_revTPSS = -3037.62263285
Exp3_CuL_B3LYP   = -3037.90447495
Exp3_CuL_M06     = -3037.87451339
Exp3_CuL_xTB     = -48.936253441819


# Proposed structure's data
C1_L_B3LYP = -1492.92972617
C2_L_B3LYP = -1492.95064231
C3_L_B3LYP = -1492.94391584
C4_L_B3LYP = -1492.94946374
C5_L_B3LYP = -1186.14957822
C6_L_B3LYP = -1186.15919731
C7_L_B3LYP = -1186.15044843
C8_L_B3LYP = -1186.16056202

C1_CuL_B3LYP = -3133.02320145
C2_CuL_B3LYP = -3133.03562077
C3_CuL_B3LYP = -3133.03564448
C4_CuL_B3LYP = -3133.04438287
C5_CuL_B3LYP = -2826.23180742
C6_CuL_B3LYP = -2826.23807044
C7_CuL_B3LYP = -2826.23860880
C8_CuL_B3LYP = -2826.23983004

C1_L_OH_B3LYP =  DG2_L_B3LYP
C2_L_OH_B3LYP = -1793.51106880
C3_L_OH_B3LYP = -1643.22622583
C4_L_OH_B3LYP = -1793.51770080
C7_L_OH_B3LYP = -1261.29286477

C1_CuL_OH_B3LYP =  DG2_CuL_B3LYP
C2_CuL_OH_B3LYP = -3433.60046421
C3_CuL_OH_B3LYP = -3283.31867557
C4_CuL_OH_B3LYP = -3433.61192575
C7_CuL_OH_B3LYP = -2901.38021072

# Methods and functional analysis

* Calculation of $\log \beta$ of the reference complexes (rtsc [4], gtsc [5], ptsm [6]).
* Calculation of $\log \beta$ of synthesized complexes (DU, DG, DGB).

## Reference complexes analysis

### Monomer and cluster methods

In [ ]:
#@title **Reference complex: rtsc (Exp1)**
#@markdown Stability constant of the **rtsc complex** ($\log \beta = 18.46$) calculated using xTB, revTPSS, B3LYP, and M06.
functional = ['xTB','revTPSS', 'B3LYP', 'M06']
var = ['Exp1_L_', 'Cu_H2O_', 'Exp1_CuL_', 'H2O_', 'H2O_6_']
method = ['monomer','cluster']

for m in method:
  print('Method: ',m)
  for f in functional:
    args = [globals()[v + f] for v in var] + [6, T, m]
    a, b = ConstanteK(*args)
    error = (a - 18.46)/18.46 * 100
    print(f'{f:<10}: log β = {a:4.4}', ' % error =', round(error))
  print('----------------------------------------')

In [ ]:
#@title **Reference complex: gtsc (Exp2)**
#@markdown Stability constant of the **gtsc complex** ($\log \beta = 20.65$) calculated using xTB, revTPSS, B3LYP, and M06.
functional = ['xTB','revTPSS', 'B3LYP', 'M06']
var = ['Exp2_L_', 'Cu_H2O_', 'Exp2_CuL_', 'H2O_', 'H2O_6_']
method = ['monomer','cluster']

for m in method:
  print('Method: ',m)
  for f in functional:
    args = [globals()[v + f] for v in var] + [6, T, m]
    a, b = ConstanteK(*args)
    error = (a - 20.65)/20.65 * 100
    print(f'{f:<10}: log β = {a:4.4}', ' % error =', round(error))
  print('----------------------------------------')

In [ ]:
#@title **Reference complex: ptsm (Exp3)**
#@markdown Stability constant of the **ptsm complex** ($\log \beta = 21.79$) is calculated using xTB, revTPSS, B3LYP, and M06.
functional = ['xTB','revTPSS', 'B3LYP', 'M06']
var = ['Exp3_L_', 'Cu_H2O_', 'Exp3_CuL_', 'H2O_', 'H2O_6_']
method = ['monomer','cluster']

for m in method:
  print('Method: ',m)
  for f in functional:
    args = [globals()[v + f] for v in var] + [6, T, m]
    a, b = ConstanteK(*args)
    error = (a - 21.79)/21.79 * 100
    print(f'{f:<10}: log β = {a:4.4}', ' % error =', round(error))
  print('----------------------------------------')

### Hess method

In [ ]:
#@title **Reference Complexes**
#@markdown Stability constants calculated using xTB, revTPSS, B3LYP, and M06, performing a cross-validation using 4 complexes reported in the literature.
#@markdown The error is quantified by the mean absolute error: $$\text{MAE} = \frac{1}{N}\sum_i|\log \beta_i^\text{calc} - \log \beta_i^\text{exp}|$$
#@markdown In addition, the mean squared error was also computed: $$\text{MSE} = \frac{1}{N}\sum_i(\log \beta_i^\text{calc} - \log \beta_i^\text{exp})^2$$

functional = ['xTB','revTPSS', 'B3LYP', 'M06']
complejos_exp = ['Exp1', 'Exp2', 'Exp3']
complejos_label = ['rtsc', 'gtsc','ptsm']
logK_exp = np.array([18.46, 20.65, 21.79])

# Loop for leave-one-out cross-validation
for f in functional:
  # Initialize lists to store errors
  mae_list = []
  mse_list = []
  print('Functional: ',f)
  for i in range(len(logK_exp)):
      logK_ref = logK_exp[i]
      logK_pred = []

      for j in range(len(logK_exp)):
        if j != i:
          var = [complejos_exp[j]+'_L_', 'Cu_H2O_', complejos_exp[j]+'_CuL_', 'H2O_', 'H2O_6_']
          var_ref = [complejos_exp[i]+'_L_', complejos_exp[i]+'_CuL_']
          args = [globals()[v + f] for v in var] + [6, T, 'Hess']+ [globals()[i + f] for i in var_ref] + [logK_exp[i]]
          a , _ = ConstanteK(*args)
          logK_pred.append(a)
        else:
          logK_pred.append(np.nan)

      # Filter NaN values and compare with real data
      logK_pred = np.array(logK_pred)
      mask = ~np.isnan(logK_pred)  # Mask to avoid the reference value
      real_indices = np.where(mask)[0]
      logK_real = logK_exp[mask]
      logK_pred_filtered = logK_pred[mask]

      # Calculate errors
      mae = np.mean(np.abs(logK_pred_filtered - logK_real))
      mse = np.mean((logK_pred_filtered - logK_real) ** 2)


      # Store errors
      mae_list.append(mae)
      mse_list.append(mse)

      # Print values
      print(f"Using {complejos_label[i]} log β = {logK_ref:.2f} as a reference:")
      for idx, (pred, real) in zip(real_indices, zip(logK_pred_filtered, logK_real)):
          print(f"  Predicted {complejos_label[idx]} log β = {pred:.2f}, Real {complejos_label[idx]} log β = {real:.2f}, % error = {abs(pred - real)/real*100:.2f}")
      print("-" * 70)

  # Average errors over all iterations
  final_mae = np.mean(mae_list)
  final_mse = np.mean(mse_list)

  print(f"Final MAE: {final_mae:.2f}, with {f}")
  print(f"Final MSE: {final_mse:.2f}, with {f}")
  print("\n")

## Synthesized complexes analysis

In [ ]:
#@title **Synthesized Complexes**
#@markdown Stability constants of the **synthesized complexes** calculated using the monomer method (relative stability info) and the Hess method (greater accuracy).

functional = ['B3LYP']
complejos = ['DU1', 'DU2', 'DU3', 'DG1', 'DG2', 'DGB']
complejos_label = ['rtsc', 'gtsc','ptsm']
complejos_exp = ['Exp1', 'Exp2', 'Exp3']
logK_exp = np.array([18.46, 20.65, 21.79])
metodo = ['monomer','Hess']

print('Method: Monomer')
for c in complejos:
  print('Synthesized complex:', c)
  var = [c+'_L_', 'Cu_H2O_', c+'_CuL_', 'H2O_', 'H2O_6_']
  for f in functional:
    args = [globals()[v + f] for v in var] + [6, T, 'monomer']
    a,_ = ConstanteK(*args)
    print(f'  {f:<6}: log β = {a:.2f}')
  print('-'*50)

print('\n\n')
print('Method: Hess')
print('\n')
for c in range(len(complejos)):
  print(complejos[c])
  var = [complejos[c]+'_L_', 'Cu_H2O_', complejos[c]+'_CuL_', 'H2O_', 'H2O_6_']
  for f in functional:
    print('Calculation by ', f)
    logK_calc = []
    for j in range(len(logK_exp)):
      var_ref = [complejos_exp[j]+'_L_', complejos_exp[j]+'_CuL_']
      args = [globals()[v + f] for v in var] + [6, T, 'Hess']+ [globals()[i + f] for i in var_ref] + [logK_exp[j]]
      a , _ = ConstanteK(*args)
      logK_calc.append(a)
      print(f' Ref: {complejos_label[j]}: log β = {a:.2f}')
    print('prom = ', round(np.mean(logK_calc),2), 'σ = ', round(np.std(logK_calc, ddof = 1),20))
    print('-'*50)

# Proposed complexes analysis

In [ ]:
functional = ['B3LYP']
complejos = ['C1', 'C2', 'C3', 'C4','C5','C6','C7','C8']
complejos_label = ['rtsc', 'gtsc','ptsm']
complejos_exp = ['Exp1', 'Exp2', 'Exp3']
logK_exp = np.array([18.46, 20.65, 21.79])
metodo = ['monomer','Hess']

print('Method: Monomer')
for c in complejos:
  print('Synthesized complex:', c)
  var = [c+'_L_', 'Cu_H2O_', c+'_CuL_', 'H2O_', 'H2O_6_']
  for f in functional:
    args = [globals()[v + f] for v in var] + [6, T, 'monomer']
    a,_ = ConstanteK(*args)
    print(f'  {f:<6}: log β = {a:.2f}')
  print('-'*50)

print('\n\n')
print('Method: Hess')
print('\n')
for c in range(len(complejos)):
  print(complejos[c])
  var = [complejos[c]+'_L_', 'Cu_H2O_', complejos[c]+'_CuL_', 'H2O_', 'H2O_6_']
  for f in functional:
    print('Calculation by ', f)
    logK_calc = []
    for j in range(len(logK_exp)):
      var_ref = [complejos_exp[j]+'_L_', complejos_exp[j]+'_CuL_']
      args = [globals()[v + f] for v in var] + [6, T, 'Hess']+ [globals()[i + f] for i in var_ref] + [logK_exp[j]]
      a , _ = ConstanteK(*args)
      logK_calc.append(a)
      print(f' Ref: {complejos_label[j]}: log β = {a:.2f}')
    print('prom = ', round(np.mean(logK_calc),2), 'σ = ', round(np.std(logK_calc, ddof = 1),20))
    print('-'*50)

In [ ]:
functional = ['B3LYP']
complejos = ['C2', 'C3', 'C4','C7']
complejos_label = ['rtsc', 'gtsc','ptsm']
complejos_exp = ['Exp1', 'Exp2', 'Exp3']
logK_exp = np.array([18.46, 20.65, 21.79])
metodo = ['monomer','Hess']

print('Method: Monomer')
for c in complejos:
  print('Synthesized complex:', c)
  var = [c+'_L_OH_', 'Cu_H2O_', c+'_CuL_OH_', 'H2O_', 'H2O_6_']
  for f in functional:
    args = [globals()[v + f] for v in var] + [6, T, 'monomer']
    a,_ = ConstanteK(*args)
    print(f'  {f:<6}: log β = {a:.2f}')
  print('-'*50)

print('\n\n')
print('Method: Hess')
print('\n')
for c in range(len(complejos)):
  print(complejos[c]+'_OH')
  var = [complejos[c]+'_L_OH_', 'Cu_H2O_', complejos[c]+'_CuL_OH_', 'H2O_', 'H2O_6_']
  for f in functional:
    print('Calculation by ', f)
    logK_calc = []
    for j in range(len(logK_exp)):
      var_ref = [complejos_exp[j]+'_L_', complejos_exp[j]+'_CuL_']
      args = [globals()[v + f] for v in var] + [6, T, 'Hess']+ [globals()[i + f] for i in var_ref] + [logK_exp[j]]
      a , _ = ConstanteK(*args)
      logK_calc.append(a)
      print(f' Ref: {complejos_label[j]}: log β = {a:.2f}')
    print('prom = ', round(np.mean(logK_calc),2), 'σ = ', round(np.std(logK_calc, ddof = 1),20))
    print('-'*50)

# References
1. Vukovic, S.; Hay, B.; Bryantsev, V. *Inorganic Chemistry* **2015**, 54, 3995–4001, DOI:10.1021/acs.inorgchem.5b00264.
2. Bryantsev, V.; Diallo, M.; Goddard III, W. *The Journal of Physical Chemistry B* **2008**, 112, 9709–9719, DOI: 10.1021/jp802665d.
3. Flores, R.; Reyes-García, L.; Rodríguez-Laguna, N.; Gómez-Balderas, R. *Theoretical Chemistry Accounts* **2018**, 137, 125, DOI: 10.1007/s00214-018-2315-z.
4. Ngarivhume, T.; Díaz, A.; Cao, R.; Ortiz, M.; Sánchez, I. *Synthesis and Reactivity in Inorganic, Metal-Organic, and Nano-Metal Chemistry* **2005**, 35, 795–800, DOI: 10.1080/15533170500360180.
5. Signorella, S.; Palopoli, C.; Frutos, A.; Escandar, G.; Tanase, T.; Sala, L. *Canadian Journal of Chemistry* **1999**, 77, 1492–1497, DOI: 10.1139/v99-165.
6. Winkelmann, D.; Bermke, Y.; Petering, D. *Bioinorganic Chemistry* **1974**, 3, 261–277, DOI: 10.1016/S0006-3061(00)80074-5.